# CCSDT theory for a closed-shell reference

In this notebook we will use wicked to generate equations for the CCSDT method

In [1]:
import wicked as w

import psi4
import forte
import forte.utils
from forte import forte_options
import numpy as np
import time

In [2]:
w.reset_space()
w.add_space("o", "fermion", "occupied", ["i", "j", "k", "l", "m", "n"])
w.add_space("v", "fermion", "unoccupied", ["a", "b", "c", "d", "e", "f"])

Top = w.op("T", ["v+ o", "v+ v+ o o", "v+ v+ v+ o o o"])
Hop = w.utils.gen_op("H",1,"ov","ov") + w.utils.gen_op("H",2,"ov","ov")

In [3]:
wt = w.WickTheorem()
Hbar = w.bch_series(Hop,Top,4)
expr = wt.contract(w.rational(1), Hbar, 0, 6)
mbeq = expr.to_manybody_equation("R")

def generate_equation(mbeq, nocc, nvir):
    res_sym = f"R{'o' * nocc}{'v' * nvir}"
    code = [f"def evaluate_residual_{nocc}_{nvir}(H,T):",
        "    # contributions to the residual"]
    if nocc + nvir == 0:
        code.append("    R = 0.0")
    else:
        dims = ','.join(['nocc'] * nocc + ['nvir'] * nvir)
        code.append(f"    {res_sym} = np.zeros(({dims}))")
    for eq in mbeq["o" * nocc + "|" + "v" * nvir]:
        contraction = eq.compile("einsum")
        code.append(f'    {contraction}')
    code.append(f'    return {res_sym}')
    funct = '\n'.join(code)
    exec(funct)
    print(f'\n\n{funct}\n')
    return funct

energy_eq = generate_equation(mbeq, 0,0)
exec(energy_eq)
t1_eq = generate_equation(mbeq, 1,1)
exec(t1_eq)
t2_eq = generate_equation(mbeq, 2,2)
exec(t2_eq)
t3_eq = generate_equation(mbeq, 3,3)
exec(t3_eq)



def evaluate_residual_0_0(H,T):
    # contributions to the residual
    R = 0.0
    R += 1.000000000 * np.einsum("ai,ia->",H["vo"],T["ov"],optimize="optimal")
    R += 0.250000000 * np.einsum("abij,ijab->",H["vvoo"],T["oovv"],optimize="optimal")
    R += 0.500000000 * np.einsum("abij,jb,ia->",H["vvoo"],T["ov"],T["ov"],optimize="optimal")
    return R



def evaluate_residual_1_1(H,T):
    # contributions to the residual
    Rov = np.zeros((nocc,nvir))
    Rov += -1.000000000 * np.einsum("ij,ja->ia",H["oo"],T["ov"],optimize="optimal")
    Rov += -1.000000000 * np.einsum("bj,ja,ib->ia",H["vo"],T["ov"],T["ov"],optimize="optimal")
    Rov += 1.000000000 * np.einsum("bj,ijab->ia",H["vo"],T["oovv"],optimize="optimal")
    Rov += 1.000000000 * np.einsum("ibjk,ka,jb->ia",H["ovoo"],T["ov"],T["ov"],optimize="optimal")
    Rov += -0.500000000 * np.einsum("ibjk,jkab->ia",H["ovoo"],T["oovv"],optimize="optimal")
    Rov += -0.500000000 * np.einsum("bcjk,ja,ikbc->ia",H["vvoo"],T["ov"],T["oovv"],opt

```python
def evaluate_residual_0_0(H,T):
    # contributions to the residual
    R = 0.0
    R += 1.000000000 * np.einsum("ai,ia->",H["vo"],T["ov"])
    R += 0.500000000 * np.einsum("abij,ia,jb->",H["vvoo"],T["ov"],T["ov"])
    R += 0.250000000 * np.einsum("abij,ijab->",H["vvoo"],T["oovv"])
    return R



def evaluate_residual_1_1(H,T):
    # contributions to the residual
    Rov = np.zeros((nocc,nvir))
    Rov += 1.000000000 * np.einsum("ba,ib->ia",H["vv"],T["ov"])
    Rov += 1.000000000 * np.einsum("ia->ia",H["ov"])
    Rov += -1.000000000 * np.einsum("bj,ja,ib->ia",H["vo"],T["ov"],T["ov"])
    Rov += 1.000000000 * np.einsum("bj,ijab->ia",H["vo"],T["oovv"])
    Rov += -1.000000000 * np.einsum("ij,ja->ia",H["oo"],T["ov"])
    Rov += 1.000000000 * np.einsum("bcja,ic,jb->ia",H["vvov"],T["ov"],T["ov"])
    Rov += -0.500000000 * np.einsum("bcja,ijbc->ia",H["vvov"],T["oovv"])
    Rov += -1.000000000 * np.einsum("ibja,jb->ia",H["ovov"],T["ov"])
    Rov += 1.000000000 * np.einsum("bcjk,kc,ijab->ia",H["vvoo"],T["ov"],T["oovv"])
    Rov += 0.500000000 * np.einsum("bcjk,ic,jkab->ia",H["vvoo"],T["ov"],T["oovv"])
    Rov += -1.000000000 * np.einsum("bcjk,ka,ic,jb->ia",H["vvoo"],T["ov"],T["ov"],T["ov"])
    Rov += 0.500000000 * np.einsum("bcjk,ka,ijbc->ia",H["vvoo"],T["ov"],T["oovv"])
    Rov += 0.250000000 * np.einsum("bcjk,ijkabc->ia",H["vvoo"],T["ooovvv"])
    Rov += 1.000000000 * np.einsum("ibjk,ka,jb->ia",H["ovoo"],T["ov"],T["ov"])
    Rov += -0.500000000 * np.einsum("ibjk,jkab->ia",H["ovoo"],T["oovv"])
    return Rov



def evaluate_residual_2_2(H,T):
    # contributions to the residual
    Roovv = np.zeros((nocc,nocc,nvir,nvir))
    Roovv += -2.000000000 * np.einsum("ca,ijbc->ijab",H["vv"],T["oovv"])
    Roovv += 1.000000000 * np.einsum("cdab,ic,jd->ijab",H["vvvv"],T["ov"],T["ov"])
    Roovv += 0.500000000 * np.einsum("cdab,ijcd->ijab",H["vvvv"],T["oovv"])
    Roovv += 2.000000000 * np.einsum("icab,jc->ijab",H["ovvv"],T["ov"])
    Roovv += 1.000000000 * np.einsum("ijab->ijab",H["oovv"])
    Roovv += 2.000000000 * np.einsum("ck,ic,jkab->ijab",H["vo"],T["ov"],T["oovv"])
    Roovv += 2.000000000 * np.einsum("ck,ka,ijbc->ijab",H["vo"],T["ov"],T["oovv"])
    Roovv += 1.000000000 * np.einsum("ck,ijkabc->ijab",H["vo"],T["ooovvv"])
    Roovv += 2.000000000 * np.einsum("ik,jkab->ijab",H["oo"],T["oovv"])
    Roovv += 2.000000000 * np.einsum("cdka,kd,ijbc->ijab",H["vvov"],T["ov"],T["oovv"])
    Roovv += 4.000000000 * np.einsum("cdka,id,jkbc->ijab",H["vvov"],T["ov"],T["oovv"])
    Roovv += 2.000000000 * np.einsum("cdka,kb,ic,jd->ijab",H["vvov"],T["ov"],T["ov"],T["ov"])
    Roovv += 1.000000000 * np.einsum("cdka,kb,ijcd->ijab",H["vvov"],T["ov"],T["oovv"])
    Roovv += 1.000000000 * np.einsum("cdka,ijkbcd->ijab",H["vvov"],T["ooovvv"])
    Roovv += 4.000000000 * np.einsum("icka,kb,jc->ijab",H["ovov"],T["ov"],T["ov"])
    Roovv += -4.000000000 * np.einsum("icka,jkbc->ijab",H["ovov"],T["oovv"])
    Roovv += 2.000000000 * np.einsum("ijka,kb->ijab",H["ooov"],T["ov"])
    Roovv += 1.000000000 * np.einsum("cdkl,ld,ijkabc->ijab",H["vvoo"],T["ov"],T["ooovvv"])
    Roovv += -2.000000000 * np.einsum("cdkl,id,lc,jkab->ijab",H["vvoo"],T["ov"],T["ov"],T["oovv"])
    Roovv += -1.000000000 * np.einsum("cdkl,id,jklabc->ijab",H["vvoo"],T["ov"],T["ooovvv"])
    Roovv += 0.500000000 * np.einsum("cdkl,ic,jd,klab->ijab",H["vvoo"],T["ov"],T["ov"],T["oovv"])
    Roovv += -2.000000000 * np.einsum("cdkl,la,kd,ijbc->ijab",H["vvoo"],T["ov"],T["ov"],T["oovv"])
    Roovv += -4.000000000 * np.einsum("cdkl,la,id,jkbc->ijab",H["vvoo"],T["ov"],T["ov"],T["oovv"])
    Roovv += -1.000000000 * np.einsum("cdkl,la,ijkbcd->ijab",H["vvoo"],T["ov"],T["ooovvv"])
    Roovv += 1.000000000 * np.einsum("cdkl,ka,lb,ic,jd->ijab",H["vvoo"],T["ov"],T["ov"],T["ov"],T["ov"])
    Roovv += 0.500000000 * np.einsum("cdkl,ka,lb,ijcd->ijab",H["vvoo"],T["ov"],T["ov"],T["oovv"])
    Roovv += 1.000000000 * np.einsum("cdkl,ijad,klbc->ijab",H["vvoo"],T["oovv"],T["oovv"])
    Roovv += 2.000000000 * np.einsum("cdkl,ikac,jlbd->ijab",H["vvoo"],T["oovv"],T["oovv"])
    Roovv += 0.250000000 * np.einsum("cdkl,klab,ijcd->ijab",H["vvoo"],T["oovv"],T["oovv"])
    Roovv += 1.000000000 * np.einsum("cdkl,ilab,jkcd->ijab",H["vvoo"],T["oovv"],T["oovv"])
    Roovv += 2.000000000 * np.einsum("ickl,lc,jkab->ijab",H["ovoo"],T["ov"],T["oovv"])
    Roovv += 1.000000000 * np.einsum("ickl,jc,klab->ijab",H["ovoo"],T["ov"],T["oovv"])
    Roovv += 4.000000000 * np.einsum("ickl,la,jkbc->ijab",H["ovoo"],T["ov"],T["oovv"])
    Roovv += 2.000000000 * np.einsum("ickl,ka,lb,jc->ijab",H["ovoo"],T["ov"],T["ov"],T["ov"])
    Roovv += 1.000000000 * np.einsum("ickl,jklabc->ijab",H["ovoo"],T["ooovvv"])
    Roovv += 1.000000000 * np.einsum("ijkl,ka,lb->ijab",H["oooo"],T["ov"],T["ov"])
    Roovv += 0.500000000 * np.einsum("ijkl,klab->ijab",H["oooo"],T["oovv"])
    return Roovv



def evaluate_residual_3_3(H,T):
    # contributions to the residual
    Rooovvv = np.zeros((nocc,nocc,nocc,nvir,nvir,nvir))
    Rooovvv += 3.000000000 * np.einsum("da,ijkbcd->ijkabc",H["vv"],T["ooovvv"])
    Rooovvv += 9.000000000 * np.einsum("deab,ie,jkcd->ijkabc",H["vvvv"],T["ov"],T["oovv"])
    Rooovvv += 1.500000000 * np.einsum("deab,ijkcde->ijkabc",H["vvvv"],T["ooovvv"])
    Rooovvv += -9.000000000 * np.einsum("idab,jkcd->ijkabc",H["ovvv"],T["oovv"])
    Rooovvv += -3.000000000 * np.einsum("dl,id,jklabc->ijkabc",H["vo"],T["ov"],T["ooovvv"])
    Rooovvv += -3.000000000 * np.einsum("dl,la,ijkbcd->ijkabc",H["vo"],T["ov"],T["ooovvv"])
    Rooovvv += 9.000000000 * np.einsum("dl,ilab,jkcd->ijkabc",H["vo"],T["oovv"],T["oovv"])
    Rooovvv += -3.000000000 * np.einsum("il,jklabc->ijkabc",H["oo"],T["ooovvv"])
    Rooovvv += -3.000000000 * np.einsum("dela,le,ijkbcd->ijkabc",H["vvov"],T["ov"],T["ooovvv"])
    Rooovvv += 9.000000000 * np.einsum("dela,ie,jklbcd->ijkabc",H["vvov"],T["ov"],T["ooovvv"])
    Rooovvv += -9.000000000 * np.einsum("dela,id,je,klbc->ijkabc",H["vvov"],T["ov"],T["ov"],T["oovv"])
    Rooovvv += 18.000000000 * np.einsum("dela,lb,ie,jkcd->ijkabc",H["vvov"],T["ov"],T["ov"],T["oovv"])
    Rooovvv += 3.000000000 * np.einsum("dela,lb,ijkcde->ijkabc",H["vvov"],T["ov"],T["ooovvv"])
    Rooovvv += -18.000000000 * np.einsum("dela,ijbe,klcd->ijkabc",H["vvov"],T["oovv"],T["oovv"])
    Rooovvv += -4.500000000 * np.einsum("dela,ilbc,jkde->ijkabc",H["vvov"],T["oovv"],T["oovv"])
    Rooovvv += -18.000000000 * np.einsum("idla,jd,klbc->ijkabc",H["ovov"],T["ov"],T["oovv"])
    Rooovvv += -18.000000000 * np.einsum("idla,lb,jkcd->ijkabc",H["ovov"],T["ov"],T["oovv"])
    Rooovvv += -9.000000000 * np.einsum("idla,jklbcd->ijkabc",H["ovov"],T["ooovvv"])
    Rooovvv += -9.000000000 * np.einsum("ijla,klbc->ijkabc",H["ooov"],T["oovv"])
    Rooovvv += 9.000000000 * np.einsum("delm,me,ilab,jkcd->ijkabc",H["vvoo"],T["ov"],T["oovv"],T["oovv"])
    Rooovvv += 3.000000000 * np.einsum("delm,ie,md,jklabc->ijkabc",H["vvoo"],T["ov"],T["ov"],T["ooovvv"])
    Rooovvv += 4.500000000 * np.einsum("delm,ie,lmab,jkcd->ijkabc",H["vvoo"],T["ov"],T["oovv"],T["oovv"])
    Rooovvv += 18.000000000 * np.einsum("delm,ie,jmab,klcd->ijkabc",H["vvoo"],T["ov"],T["oovv"],T["oovv"])
    Rooovvv += 1.500000000 * np.einsum("delm,id,je,klmabc->ijkabc",H["vvoo"],T["ov"],T["ov"],T["ooovvv"])
    Rooovvv += -1.500000000 * np.einsum("delm,imde,jklabc->ijkabc",H["vvoo"],T["oovv"],T["ooovvv"])
    Rooovvv += 0.750000000 * np.einsum("delm,ijde,klmabc->ijkabc",H["vvoo"],T["oovv"],T["ooovvv"])
    Rooovvv += 3.000000000 * np.einsum("delm,ma,le,ijkbcd->ijkabc",H["vvoo"],T["ov"],T["ov"],T["ooovvv"])
    Rooovvv += -9.000000000 * np.einsum("delm,ma,ie,jklbcd->ijkabc",H["vvoo"],T["ov"],T["ov"],T["ooovvv"])
    Rooovvv += 9.000000000 * np.einsum("delm,ma,id,je,klbc->ijkabc",H["vvoo"],T["ov"],T["ov"],T["ov"],T["oovv"])
    Rooovvv += 18.000000000 * np.einsum("delm,ma,ijbe,klcd->ijkabc",H["vvoo"],T["ov"],T["oovv"],T["oovv"])
    Rooovvv += 4.500000000 * np.einsum("delm,ma,ilbc,jkde->ijkabc",H["vvoo"],T["ov"],T["oovv"],T["oovv"])
    Rooovvv += 9.000000000 * np.einsum("delm,la,mb,ie,jkcd->ijkabc",H["vvoo"],T["ov"],T["ov"],T["ov"],T["oovv"])
    Rooovvv += 1.500000000 * np.einsum("delm,la,mb,ijkcde->ijkabc",H["vvoo"],T["ov"],T["ov"],T["ooovvv"])
    Rooovvv += -1.500000000 * np.einsum("delm,lmae,ijkbcd->ijkabc",H["vvoo"],T["oovv"],T["ooovvv"])
    Rooovvv += 9.000000000 * np.einsum("delm,imae,jklbcd->ijkabc",H["vvoo"],T["oovv"],T["ooovvv"])
    Rooovvv += -4.500000000 * np.einsum("delm,ijae,klmbcd->ijkabc",H["vvoo"],T["oovv"],T["ooovvv"])
    Rooovvv += 0.750000000 * np.einsum("delm,lmab,ijkcde->ijkabc",H["vvoo"],T["oovv"],T["ooovvv"])
    Rooovvv += -4.500000000 * np.einsum("delm,imab,jklcde->ijkabc",H["vvoo"],T["oovv"],T["ooovvv"])
    Rooovvv += -3.000000000 * np.einsum("idlm,md,jklabc->ijkabc",H["ovoo"],T["ov"],T["ooovvv"])
    Rooovvv += 3.000000000 * np.einsum("idlm,jd,klmabc->ijkabc",H["ovoo"],T["ov"],T["ooovvv"])
    Rooovvv += 18.000000000 * np.einsum("idlm,ma,jd,klbc->ijkabc",H["ovoo"],T["ov"],T["ov"],T["oovv"])
    Rooovvv += 9.000000000 * np.einsum("idlm,ma,jklbcd->ijkabc",H["ovoo"],T["ov"],T["ooovvv"])
    Rooovvv += -9.000000000 * np.einsum("idlm,la,mb,jkcd->ijkabc",H["ovoo"],T["ov"],T["ov"],T["oovv"])
    Rooovvv += -4.500000000 * np.einsum("idlm,lmab,jkcd->ijkabc",H["ovoo"],T["oovv"],T["oovv"])
    Rooovvv += -18.000000000 * np.einsum("idlm,jmab,klcd->ijkabc",H["ovoo"],T["oovv"],T["oovv"])
    Rooovvv += 9.000000000 * np.einsum("ijlm,ma,klbc->ijkabc",H["oooo"],T["ov"],T["oovv"])
    Rooovvv += 1.500000000 * np.einsum("ijlm,klmabc->ijkabc",H["oooo"],T["ooovvv"])
    return Rooovvv
```

## Compute the Hartree–Fock and MP2 energy

In [4]:
# setup xyz geometry for linear H4
geometry = """
H 0.0 0.0 0.0
H 0.0 0.0 1.0
H 0.0 0.0 2.0
H 0.0 0.0 3.0
H 0.0 0.0 4.0
H 0.0 0.0 5.1
symmetry c1
"""

(Escf, psi4_wfn) = forte.utils.psi4_scf(geometry,
                                        basis='sto-3g',
                                        reference='rhf',
                                        options={'E_CONVERGENCE' : 1.e-12})

## Prepare integrals for Forte

In [5]:
# Define the orbital spaces
mo_spaces = {'RESTRICTED_DOCC': [3],'RESTRICTED_UOCC': [3]}

# pass Psi4 options to Forte
options = psi4.core.get_options()
options.set_current_module('FORTE')
forte_options.get_options_from_psi4(options)

# Grab the number of MOs per irrep
nmopi = psi4_wfn.nmopi()
# Grab the point group symbol (e.g. "C2V")
point_group = psi4_wfn.molecule().point_group().symbol()
# create a MOSpaceInfo object
mo_space_info = forte.make_mo_space_info_from_map(nmopi, point_group,mo_spaces, [])
# make a ForteIntegral object
ints = forte.make_ints_from_psi4(psi4_wfn, forte_options, mo_space_info)

## Define orbital spaces and dimensions

In [6]:
occmos = mo_space_info.corr_absolute_mo('RESTRICTED_DOCC')
virmos = mo_space_info.corr_absolute_mo('RESTRICTED_UOCC')
allmos = mo_space_info.corr_absolute_mo('CORRELATED')
nocc = 2 * len(occmos)
nvir = 2 * len(virmos)

## Build the Fock matrix and the zeroth-order Fock matrix

In [7]:
H = {'oo': forte.spinorbital_fock(ints,occmos, occmos,occmos),
     'vv': forte.spinorbital_fock(ints,virmos, virmos,occmos),
     'ov': forte.spinorbital_fock(ints,occmos, virmos,occmos),
     'vo': forte.spinorbital_fock(ints,occmos, virmos,occmos),     
     'oovv' : forte.spinorbital_tei(ints,occmos,occmos,virmos,virmos),
     'ooov' : forte.spinorbital_tei(ints,occmos,occmos,occmos,virmos),
     'vvvv' : forte.spinorbital_tei(ints,virmos,virmos,virmos,virmos),
     'vvoo' : forte.spinorbital_tei(ints,virmos,virmos,occmos,occmos),
     'ovov' : forte.spinorbital_tei(ints,occmos,virmos,occmos,virmos),
     'ovvv' : forte.spinorbital_tei(ints,occmos,virmos,virmos,virmos),
     'vvov' : forte.spinorbital_tei(ints,virmos,virmos,occmos,virmos),
     'ovoo' : forte.spinorbital_tei(ints,occmos,virmos,occmos,occmos),
     'oooo' : forte.spinorbital_tei(ints,occmos,occmos,occmos,occmos)}

## Build the MP denominators

In [8]:
fo = np.diag(H['oo'])
fv = np.diag(H['vv'])

D = {}

d1 = np.zeros((nocc,nvir))
for i in range(nocc):
    for a in range(nvir):
        si = i % 2
        sa = a % 2
        if si == sa:
            d1[i][a] = 1.0 / (fo[i] - fv[a])
D['ov'] = d1
            
                    
d2 = np.zeros((nocc,nocc,nvir,nvir))
for i in range(nocc):
    for j in range(nocc):
        for a in range(nvir):
            for b in range(nvir):
                si = i % 2
                sj = j % 2
                sa = a % 2
                sb = b % 2
                if si == sj == sa == sb:
                    d2[i][j][a][b] = 1.0 / (fo[i] + fo[j] - fv[a] - fv[b])
                if si == sa and sj == sb and si != sj:
                    d2[i][j][a][b] = 1.0 / (fo[i] + fo[j] - fv[a] - fv[b])
                if si == sb and sj == sa and si != sj:
                    d2[i][j][a][b] = 1.0 / (fo[i] + fo[j] - fv[a] - fv[b])                    
D['oovv'] = d2

d3 = np.zeros((nocc,nocc,nocc,nvir,nvir,nvir))
for i in range(nocc):
    for j in range(nocc):
        for k in range(nocc):
            for a in range(nvir):
                for b in range(nvir):
                    for c in range(nvir):
                        si = i % 2
                        sj = j % 2
                        sk = k % 2
                        sa = a % 2
                        sb = b % 2
                        sc = c % 2
                        d3[i][j][k][a][b][c] = 1.0 / (fo[i] + fo[j] + fo[k]- fv[a] - fv[b] - fv[c])
D['ooovvv'] = d3

In [9]:
# Compute the MP2 correlation energy
Emp2 = 0.0
for i in range(nocc):
    for j in range(nocc):
        for a in range(nvir):
            for b in range(nvir):
                Emp2 += 0.25 * H["oovv"][i][j][a][b] ** 2 / (fo[i] + fo[j] - fv[a] - fv[b])
print(f"MP2 corr. energy: {Emp2:.12f} Eh")

MP2 corr. energy: -0.066236921094 Eh


In [10]:
def antisymmetrize_residual_2_2(Roovv):
    # antisymmetrize the residual
    Roovv_anti = np.zeros((nocc,nocc,nvir,nvir))
    Roovv_anti += np.einsum("ijab->ijab",Roovv)
    Roovv_anti -= np.einsum("ijab->jiab",Roovv)
    Roovv_anti -= np.einsum("ijab->ijba",Roovv)
    Roovv_anti += np.einsum("ijab->jiba",Roovv)    
    return Roovv_anti

def antisymmetrize_residual_3_3(Rooovvv):
    # antisymmetrize the residual
    Rooovvv_anti = np.zeros((nocc,nocc,nocc,nvir,nvir,nvir))
    Rooovvv_anti += +1 * np.einsum("ijkabc->ijkabc",Rooovvv)
    Rooovvv_anti += -1 * np.einsum("ijkabc->ijkacb",Rooovvv)
    Rooovvv_anti += -1 * np.einsum("ijkabc->ijkbac",Rooovvv)
    Rooovvv_anti += +1 * np.einsum("ijkabc->ijkbca",Rooovvv)
    Rooovvv_anti += +1 * np.einsum("ijkabc->ijkcab",Rooovvv)
    Rooovvv_anti += -1 * np.einsum("ijkabc->ijkcba",Rooovvv)
    Rooovvv_anti += -1 * np.einsum("ijkabc->ikjabc",Rooovvv)
    Rooovvv_anti += +1 * np.einsum("ijkabc->ikjacb",Rooovvv)
    Rooovvv_anti += +1 * np.einsum("ijkabc->ikjbac",Rooovvv)
    Rooovvv_anti += -1 * np.einsum("ijkabc->ikjbca",Rooovvv)
    Rooovvv_anti += -1 * np.einsum("ijkabc->ikjcab",Rooovvv)
    Rooovvv_anti += +1 * np.einsum("ijkabc->ikjcba",Rooovvv)
    Rooovvv_anti += -1 * np.einsum("ijkabc->jikabc",Rooovvv)
    Rooovvv_anti += +1 * np.einsum("ijkabc->jikacb",Rooovvv)
    Rooovvv_anti += +1 * np.einsum("ijkabc->jikbac",Rooovvv)
    Rooovvv_anti += -1 * np.einsum("ijkabc->jikbca",Rooovvv)
    Rooovvv_anti += -1 * np.einsum("ijkabc->jikcab",Rooovvv)
    Rooovvv_anti += +1 * np.einsum("ijkabc->jikcba",Rooovvv)
    Rooovvv_anti += +1 * np.einsum("ijkabc->jkiabc",Rooovvv)
    Rooovvv_anti += -1 * np.einsum("ijkabc->jkiacb",Rooovvv)
    Rooovvv_anti += -1 * np.einsum("ijkabc->jkibac",Rooovvv)
    Rooovvv_anti += +1 * np.einsum("ijkabc->jkibca",Rooovvv)
    Rooovvv_anti += +1 * np.einsum("ijkabc->jkicab",Rooovvv)
    Rooovvv_anti += -1 * np.einsum("ijkabc->jkicba",Rooovvv)
    Rooovvv_anti += +1 * np.einsum("ijkabc->kijabc",Rooovvv)
    Rooovvv_anti += -1 * np.einsum("ijkabc->kijacb",Rooovvv)
    Rooovvv_anti += -1 * np.einsum("ijkabc->kijbac",Rooovvv)
    Rooovvv_anti += +1 * np.einsum("ijkabc->kijbca",Rooovvv)
    Rooovvv_anti += +1 * np.einsum("ijkabc->kijcab",Rooovvv)
    Rooovvv_anti += -1 * np.einsum("ijkabc->kijcba",Rooovvv)
    Rooovvv_anti += -1 * np.einsum("ijkabc->kjiabc",Rooovvv)
    Rooovvv_anti += +1 * np.einsum("ijkabc->kjiacb",Rooovvv)
    Rooovvv_anti += +1 * np.einsum("ijkabc->kjibac",Rooovvv)
    Rooovvv_anti += -1 * np.einsum("ijkabc->kjibca",Rooovvv)
    Rooovvv_anti += -1 * np.einsum("ijkabc->kjicab",Rooovvv)
    Rooovvv_anti += +1 * np.einsum("ijkabc->kjicba",Rooovvv)
    return Rooovvv_anti

def update_amplitudes(T,R,d):
    T['ov'] += np.einsum("ia,ia->ia",R['ov'],D['ov'])
    T['oovv'] += np.einsum("ijab,ijab->ijab",R['oovv'],D['oovv'])
    T['ooovvv'] += np.einsum("ijkabc,ijkabc->ijkabc",R['ooovvv'],D['ooovvv'])

In [11]:
ref_CCSDT = -0.108354659115 # from forte sparse implementation

T = {}
T["ov"] = np.zeros((nocc,nvir))
T["oovv"] = np.zeros((nocc,nocc,nvir,nvir))
T["ooovvv"] = np.zeros((nocc,nocc,nocc,nvir,nvir,nvir))

header = "Iter.    Corr. energy       |R|       "
print("-" * len(header))
print(header)
print("-" * len(header))

start = time.perf_counter()

maxiter = 100
for i in range(maxiter):
    R = {}
    Ewicked = float(evaluate_residual_0_0(H,T))
    R['ov'] = evaluate_residual_1_1(H,T)
    Roovv = evaluate_residual_2_2(H,T)
    R['oovv'] = antisymmetrize_residual_2_2(Roovv)
    Rooovvv = evaluate_residual_3_3(H,T)
    R['ooovvv'] = antisymmetrize_residual_3_3(Rooovvv)    

    update_amplitudes(T,R,D)

    # check for convergence
    norm_R = np.sqrt(np.linalg.norm(R['ov'])**2 + np.linalg.norm(R['oovv'])**2 + np.linalg.norm(R['ooovvv'])**2)
    print(f"{i:3d}    {Ewicked:+.12f}  {norm_R:e}")     
    if norm_R < 1.0e-9:
        break
        
        
end = time.perf_counter()
t = end - start           
            
print("-" * len(header))      
print(f"CCSDT correlation energy: {Ewicked:+.12f} [Eh]")
print(f"Error:                    {Ewicked - ref_CCSDT:+.12e} [Eh]")
print(f"Timing:                  {t:+.12e} [s]")

--------------------------------------
Iter.    Corr. energy       |R|       
--------------------------------------
  0    +0.000000000000  7.023516e-01
  1    -0.066236921094  3.139687e-01
  2    -0.088946436431  1.624509e-01
  3    -0.098954156371  8.721897e-02
  4    -0.103462348907  5.024723e-02
  5    -0.105686843956  3.132756e-02
  6    -0.106818323730  2.102629e-02
  7    -0.107430856119  1.474433e-02
  8    -0.107776281266  1.047404e-02
  9    -0.107981644688  7.475319e-03
 10    -0.108108186331  5.316980e-03
 11    -0.108189078559  3.774804e-03
 12    -0.108242001907  2.670743e-03
 13    -0.108277364080  1.886525e-03
 14    -0.108301292128  1.329956e-03
 15    -0.108317659771  9.366991e-04
 16    -0.108328927492  6.590403e-04
 17    -0.108336726279  4.634392e-04
 18    -0.108342141451  3.257105e-04
 19    -0.108345911737  2.288417e-04
 20    -0.108348541166  1.607317e-04
 21    -0.108350377551  1.128712e-04
 22    -0.108351661252  7.924690e-05
 23    -0.108352559307  5.563207